In [1]:
import requests
from requests_html import HTMLSession
from requests_html import AsyncHTMLSession
from bs4 import BeautifulSoup
import pprint

URL = 'https://www.monster.com/jobs/search/?q=Software-Developer&where=Australia'
# URL2 = 'https://tabs.ultimate-guitar.com/tab/misc-soundtrack/a-star-is-born-shallow-chords-2488086'
# page = requests.get(URL2)

# soup = BeautifulSoup(page.content, 'html.parser')
# #pp.pprint(page.content)
# #soup.body.contents
# # for tag in soup.body.children:
# #     print(tag.name)
# #soup.find_all(class_="_2u0o0")
# x = soup.body
# for check in x.contents:
#     print(check,"\n\n")
# #x.find_all(class_="_22Bzv")
# y = x.find_all(class_='js-page js-global-wrapper')[0]
# y.find_all("div")

In [2]:
# session = HTMLSession()
# r = session.get('http://python-requests.org/')

# r.html.render()

asession = AsyncHTMLSession()
r = await asession.get('https://tabs.ultimate-guitar.com/tab/misc-soundtrack/a-star-is-born-shallow-chords-2488086',timeout=20)
#r.html.render(timeout=20)
site = await r.html.arender()
soup = BeautifulSoup(r.html.html, features='lxml')
body = soup.find(class_='js-page js-global-wrapper')
patterns = body.find_all(class_='_2u0o0')
patterns[0].contents

[<div class="_31jmH _2xk1x _1yupz"></div>,
 <div class="_31jmH _2xk1x"></div>,
 <div class="_31jmH _2xk1x _1yupz"></div>,
 <div class="_31jmH _2xk1x _3N-at"></div>,
 <div class="_31jmH _2xk1x _1yupz"></div>,
 <div class="_31jmH _2xk1x _3N-at"></div>,
 <div class="_31jmH _2xk1x"></div>,
 <div class="_31jmH _2xk1x _3N-at"></div>,
 <div class="_31jmH _2xk1x"></div>,
 <div class="_31jmH _2xk1x"></div>,
 <div class="_31jmH _2xk1x"></div>,
 <div class="_31jmH _2xk1x"></div>,
 <div class="_31jmH _2xk1x"></div>,
 <div class="_31jmH _2xk1x"></div>,
 <div class="_31jmH _2xk1x"></div>,
 <div class="_31jmH _2xk1x"></div>,
 <div class="_31jmH _2xk1x _1yupz"></div>,
 <div class="_31jmH _2xk1x"></div>,
 <div class="_31jmH _2xk1x _1yupz"></div>,
 <div class="_31jmH _2xk1x _3N-at"></div>,
 <div class="_31jmH _2xk1x"></div>,
 <div class="_31jmH _2xk1x _3N-at"></div>,
 <div class="_31jmH _2xk1x _1yupz"></div>,
 <div class="_31jmH _2xk1x _3N-at"></div>,
 <div class="_31jmH _2xk1x _1yupz"></div>,
 <div cla